In [2]:
import openai
import pandas as pd
import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load environment variables
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

source_name = "data0"
sample_size = "100"

# Load your dataframe
df = pd.read_csv(f'../Data/1000_record_chunks/1000_chunk_1.csv')  # Replace with the actual path to your dataframe
df = df.sample(int(sample_size))

# Set up OpenAI API key
openai.api_key = OPEN_AI_API_KEY



def get_gpt_explanation(buggy_code, fixed_code):
    """
    Function to get a concise, one-paragraph explanation using gpt-4o-mini of why the fixed code is correct compared to the buggy code.
    """
    # Use double curly braces {{ }} to escape literal curly braces in f-strings
    prompt = f"""You are a senior software engineer explaining a bug fix to a junior developer. 
    Your task is to provide a concise explanation of the changes made to fix the bug in the provided code snippets. Follow these guidelines:

    ### Guidelines:
    1. **Bug Identification**: Describe the error in the original code, including its type (e.g., logic error, runtime error) and its impact.
    2. **Problem Analysis**: Explain why the bug is problematic and under what conditions it causes issues.
    3. **Fix Explanation**: Describe the changes in the fixed code and how they address the issue.
    4. **Justification**: Justify why the fix is necessary and how it resolves the problem.
    5. **Improvement Highlight**: Summarize how the fix improves code reliability, functionality, or performance.

    ### Constraints:
    - The explanation must be **concise**, using no more than **100 words**.
    - The explanation should consist of **exactly three sentences**:
      1. Why the buggy code is incorrect.
      2. What changes were made in the fixed code and why they are correct.
      3. How the fixed code improves upon the buggy code.
    - Do not include meta-descriptions such as "This is a three-sentence explanation" or mention word counts in the response.
    - Ensure your explanation aligns with the code context and focuses solely on technical details relevant to the fix.

    ### Example Explanations:

    #### Example 1:
    **Buggy Code:**
    @Override protected void afterTests(){{
        try {{
            context.shutdown();
        }}
        catch (Exception e) {{
            throw new RuntimeException("String_Node_Str", e);
        }}
        super.afterTests();
    }}

    **Fixed Code:**
    @Override protected void afterTests(){{
        try {{
            context.shutdown();
        }}
        catch (Exception e) {{
            throw new RuntimeException("String_Node_Str", e);
        }}
    }}

    **Explanation:**
    The bug in the original code is the unconditional call to `super.afterTests()`, which executes even if `context.shutdown()` fails, risking inconsistent state. 
    The fixed code removes this call, ensuring `super.afterTests()` is not invoked when an exception occurs, preventing potential errors. 
    This fix ensures predictable cleanup behavior, improving code reliability.

    #### Example 2:
    **Buggy Code:**
    private void updateTreeView(Tree tree){{
        Iterator it = tree.getDepthFirstIterator(false);
        while (it.hasNext()) {{
            ((Tree<JsonTreeNode>) it.next()).setExpanded(true);
        }}
        editorTreeView.setModel(tree.copy());
    }}

    **Fixed Code:**
    private void updateTreeView(JsonTree tree){{
        JsonTree fixedTree = JsonTreeConverter.serialize(JsonTreeConverter.deserialize(tree));
        Iterator it = fixedTree.getDepthFirstIterator(false);
        while (it.hasNext()) {{
            ((JsonTree) it.next()).setExpanded(true);
        }}
        editorTreeView.setModel(fixedTree.copy());
    }}

    **Explanation:**
    The original code has a bug where it improperly casts a generic Tree to `Tree<JsonTreeNode>`, which can cause runtime errors if the types don’t match. 
    The fix uses a `JsonTree` with serialization and deserialization to ensure the tree structure is correct and safe to work with. 
    This makes the code more reliable and prevents runtime type errors.

    ---

    ### Your Task:

    **Buggy Code:**
    {buggy_code}

    **Fixed Code:**
    {fixed_code}

    **Why the fixed code is correct:**"""


    # Ensure the OpenAI API call is correctly structured
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides clear, concise explanations of bug fixes for junior developers."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,  # Limit the output for brevity
        temperature=0.7
    )
    explanation = response['choices'][0]['message']['content'].strip()
    return explanation






# Apply both functions to each row in the DataFrame and store the results in new columns
df['gpt_explanation'] = df.apply(lambda row: get_gpt_explanation(row['buggy_code'], row['fixed_code']), axis=1)
df.to_csv(f'../Data/evaluations/{sample_size}_record_eval_only_gpt_4-0_mini.csv', index=False)

KeyboardInterrupt: 

In [23]:
import anthropic
import pandas as pd
import os

# Load environment variables
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

source_name = "data0"
sample_size = "100"

# Load your dataframe
df = pd.read_csv(f'../Data/evaluations/{sample_size}_record_eval_only_gpt_4-0_mini.csv')  # Replace with the actual path to your dataframe
# df = df.sample(int(sample_size))

# Initialize the Anthropic client
client = anthropic.Client(api_key=ANTHROPIC_API_KEY)

def get_anthropic_explanation(buggy_code, fixed_code):
    """
    Function to get a concise, one-paragraph explanation using claude-3-5-haiku of why the fixed code is correct compared to the buggy code.
    """
    # Use double curly braces {{ }} to escape literal curly braces in f-strings
    prompt = f"""You are a senior software engineer explaining a bug fix to a junior developer. 
    Your task is to provide a concise explanation of the changes made to fix the bug in the provided code snippets. Follow these guidelines:

    ### Guidelines:
    1. **Bug Identification**: Describe the error in the original code, including its type (e.g., logic error, runtime error) and its impact.
    2. **Problem Analysis**: Explain why the bug is problematic and under what conditions it causes issues.
    3. **Fix Explanation**: Describe the changes in the fixed code and how they address the issue.
    4. **Justification**: Justify why the fix is necessary and how it resolves the problem.
    5. **Improvement Highlight**: Summarize how the fix improves code reliability, functionality, or performance.

    ### Constraints:
    - The explanation must be **concise**, using no more than **100 words**.
    - The explanation should consist of **exactly three sentences**:
      1. Why the buggy code is incorrect.
      2. What changes were made in the fixed code and why they are correct.
      3. How the fixed code improves upon the buggy code.
    - Do not include meta-descriptions such as "This is a three-sentence explanation" or mention word counts in the response.
    - Ensure your explanation aligns with the code context and focuses solely on technical details relevant to the fix.


    ### Example Explanations:

    #### Example 1:
    **Buggy Code:**
    @Override protected void afterTests(){{
        try {{
            context.shutdown();
        }}
        catch (Exception e) {{
            throw new RuntimeException("String_Node_Str", e);
        }}
        super.afterTests();
    }}

    **Fixed Code:**
    @Override protected void afterTests(){{
        try {{
            context.shutdown();
        }}
        catch (Exception e) {{
            throw new RuntimeException("String_Node_Str", e);
        }}
    }}

    **Explanation:**
    The bug in the original code is the unconditional call to `super.afterTests()`, which executes even if `context.shutdown()` fails, risking inconsistent state. 
    The fixed code removes this call, ensuring `super.afterTests()` is not invoked when an exception occurs, preventing potential errors. 
    This fix ensures predictable cleanup behavior, improving code reliability.

    #### Example 2:
    **Buggy Code:**
    private void updateTreeView(Tree tree){{
        Iterator it = tree.getDepthFirstIterator(false);
        while (it.hasNext()) {{
            ((Tree<JsonTreeNode>) it.next()).setExpanded(true);
        }}
        editorTreeView.setModel(tree.copy());
    }}

    **Fixed Code:**
    private void updateTreeView(JsonTree tree){{
        JsonTree fixedTree = JsonTreeConverter.serialize(JsonTreeConverter.deserialize(tree));
        Iterator it = fixedTree.getDepthFirstIterator(false);
        while (it.hasNext()) {{
            ((JsonTree) it.next()).setExpanded(true);
        }}
        editorTreeView.setModel(fixedTree.copy());
    }}

    **Explanation:**
    The original code has a bug where it improperly casts a generic Tree to `Tree<JsonTreeNode>`, which can cause runtime errors if the types don’t match. 
    The fix uses a `JsonTree` with serialization and deserialization to ensure the tree structure is correct and safe to work with. 
    This makes the code more reliable and prevents runtime type errors.

    ---

    ### Your Task:

    **Buggy Code:**
    {buggy_code}

    **Fixed Code:**
    {fixed_code}

    **Why the fixed code is correct:**"""
    
    # Invoke the Claude 3 model
    response = client.messages.create(
        model="claude-3-5-haiku-latest",  # Use Claude 3.5 model
        max_tokens=200,
        temperature=0.5,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Access the completion result correctly
    explanation = response.content[0].text.strip()
    return explanation


# Apply the function to each row in the DataFrame
df['anthropic_explanation'] = df.apply(lambda row: get_anthropic_explanation(row['buggy_code'], row['fixed_code']), axis=1)

df.to_csv(f'../Data/evaluations/{sample_size}_record_eval_gpt_anthropic.csv', index=False)


In [18]:
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

sample_size = "100"

df = pd.read_csv(f'../Data/evaluations/{sample_size}_record_eval_gpt_anthropic.csv')  # Replace with the actual path to your dataframe
# Function to filter sentences from the text
def filter_sentences(text):
    # Split the text into individual sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Regular expression to match the unwanted phrases
    pattern = (r'\b(?:3[-\s]?sentence|3\s?sentences|three\s?sentence|Here\s+is\s+a)\b')
    # Filter sentences that do not match the pattern
    filtered = [s for s in sentences if not re.search(pattern, s, re.IGNORECASE)]
    # Join the filtered sentences back into a single text
    return ' '.join(filtered)

def calculate_similarity(text1, text2):
    """
    Calculate cosine similarity between two pieces of text using SentenceTransformer.
    """
    # Load the pre-trained model from SentenceTransformers
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Encode the two texts
    embeddings = model.encode([text1, text2])

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    
    return similarity_score

# Apply the function to the 'anthropic_explanation' column
df['anthropic_explanation'] = df['anthropic_explanation'].apply(filter_sentences)
# df['similarity_score'] = df.apply(lambda row: calculate_similarity(row['gpt_explanation'], row['anthropic_explanation']), axis=1)

df.to_csv(f'../Data/evaluations/{sample_size}_record_eval_gpt_anthropic_new.csv', index=False)